<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.4: Sentiment Analysis
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

Based on the video tutorial **Text Classification with Machine Learning,SpaCy and Scikit(Sentiment Analysis)** by **Jesse E. Agbe (JCharis)**.

## Data Source: UCI
### UCI - Machine Learning Repository
- Center for Machine Learning and Intelligent Systems

The [**UCI Machine Learning Repository**](http://archive.ics.uci.edu/ml/about.html) is a collection of databases, domain theories, and data generators that are used by the machine learning community for the empirical analysis of machine learning algorithms.

### Dataset
- [Sentiment Labelled Sentences Data Set](http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)

### Abstract
The dataset contains sentences labelled with positive or negative sentiment.

- Data Set Characteristics: Text
- Number of Instances: 3000
- Area: N/A
- Attribute Characteristics: N/A
- Number of Attributes: N/A
- Date Donated: 2015-05-30
- Associated Tasks: Classification
- Missing Values? N/A
- Number of Web Hits: 102584

### Source
Dimitrios Kotzias dkotzias '@' ics.uci.edu

### Data Set Information
This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015

Please cite the paper if you want to use it :)

It contains sentences labelled with positive or negative sentiment.

### Format
sentence &lt;tab&gt; score &lt;newline&gt;

### Details
Score is either 1 (for positive) or 0 (for negative)

The sentences come from three different websites/fields:
- imdb.com
- amazon.com
- yelp.com

For each website, there exist **500 positive** and **500 negative** sentences. Those were selected randomly for larger datasets of reviews.

We attempted to select sentences that have a clearly positive or negative connotaton, the goal was for no neutral sentences to be selected.

For the full datasets look:

- **imdb**: Maas et. al., 2011 _Learning word vectors for sentiment analysis_
- **amazon**: McAuley et. al., 2013 _Hidden factors and hidden topics: Understanding rating dimensions with review text_
- **yelp**: [Yelp dataset challenge](http://www.yelp.com/dataset_challenge)


### Attribute Information
The attributes are text sentences, extracted from reviews of products, movies, and restaurants

### Relevant Papers
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

### Citation Request
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

## Import libraries

In [69]:
## Import Libraries
import pandas as pd

import regex as re
import spacy

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

import pickle
import sys
import os
import io
import re
from sys import path
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from string import punctuation, digits
from IPython.core.display import display, HTML
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import warnings
warnings.filterwarnings('ignore')

## Load data

Load Yelp, Amazon and Imdb Data.

Hint: Source is separated by <tab>s and has no headers.

In [14]:
# yelp_text = 'yelp_labelled.txt'
# imdb_text = 'imdb_labelled_fixed.txt'
# amazon_text = 'amazon_cells_labelled.txt'

# ANSWER

df_yelp = pd.read_table('yelp_labelled.txt', header = None, sep = '\t')
df_imdb = pd.read_table('imdb_labelled.txt', header = None, sep = '\t') # New file without double quotes
df_amazon = pd.read_table('amazon_cells_labelled.txt', header = None, sep = '\t')
# dfs = {'yelp': df_yelp, 'imdb': df_imdb, 'amazon': df_amazon}

In [25]:
frames = [df_yelp, df_imdb, df_amazon]

## Inspect the data

Check your datasets.

In [30]:
# ANSWER
df_yelp.columns

Index(['Message', 'Target'], dtype='object')

In [31]:
df_imdb.columns

Index(['Message', 'Target'], dtype='object')

In [32]:
df_amazon.columns

Index(['Message', 'Target'], dtype='object')

In [33]:
for colname in frames:
    colname.columns = ["Message","Target"] # text, sentiment #sentence, class

In [34]:
for colname in frames:
    print(colname.columns)

Index(['Message', 'Target'], dtype='object')
Index(['Message', 'Target'], dtype='object')
Index(['Message', 'Target'], dtype='object')


## Merge the data

Merge all three datasets.

In [ ]:
# ANSWER

In [35]:
keys= ['Yelp', 'IMDB', 'Amazon']

In [36]:
df=pd.concat(frames, keys=keys)

In [37]:
df.shape

(2748, 2)

In [38]:
df.head()

Message  Target
Yelp 0                           Wow... Loved this place.       1
     1                                 Crust is not good.       0
     2          Not tasty and the texture was just nasty.       0
     3  Stopped by during the late May bank holiday of...       1
     4  The selection on the menu was great and so wer...       1

## Export the data

Export merged datasets to as csv file.

In [39]:
# ANSWER
df.to_csv("Sentiment_Analysis.csv")

In [40]:
df.columns

Index(['Message', 'Target'], dtype='object')

In [41]:
df.isnull().sum()

Message    0
Target     0
dtype: int64

## Prepare the stage
- Load spaCy

In [52]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_sm")

## Prepare the text
All the text handling and preparation concerned with the changes and modifications from the raw source text to a format that will be used for the actual processing, things like:
- handle encoding
- handle extraneous and international charaters
- handle simbols
- handle metadata and embeded information
- handle repetitions (such multiple spaces or newlines)

Clean text.

In [76]:
def clean_text(text):
    # reduce multiple spaces and newlines to only one
    text = re.sub(r'(\s\s+|\n\n+)', r'\1', text)
    # remove double quotes
    text = re.sub(r'"', '', text)

    return text

In [ ]:
# ANSWER

In [78]:
df['Message'] = df['Message'].str.lower()
#remove email adress
df['Message'] = df['Message'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)
#remove IP address
df['Message'] = df['Message'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)
#remove punctaitions and special chracters
df['Message'] = df['Message'].str.replace('[^\w\s]','')
#remove numbers
df['Message'] = df['Message'].replace('\d', '', regex=True)
df['Message'] = df['Message'].apply(clean_text)


UsageError: Line magic function `%%time` not found.


In [79]:
df.sample(20)

Message  Target
IMDB   375  he is almost unbearable to watch on screen he ...       0
Amazon 204                     battery lasts only a few hours       0
       15   highly recommend for any one who has a blue to...       1
Yelp   218            which are small and not worth the price       0
       745  there was a warm feeling with the service and ...       1
Amazon 524                                        works great       1
IMDB   190  here in the wind and the lion we see a wonderf...       1
Yelp   22                                service is also cute       1
       262            the restaurant atmosphere was exquisite       1
IMDB   283  and finally after all that we get to an ending...       0
       323  watching washing machine twirling around would...       0
Amazon 541   ive missed numerous calls because of this reason       0
IMDB   109  i was left shattered from the experience of wa...       0
Amazon 909  seller shipped quickly and much cheaper than t...       1
IMDB   517  im big fan of rpg games too but this movie its...       0
Amazon 31   this product is ideal for people like me whose...       1
Yelp   284  i would definitely recommend the wings as well...       1
Amazon 383  after trying many many handsfree gadgets this ...       1
IMDB   392      i cant believe theres even a sequel to this         0
Amazon 4                                     the mic is great       1

## Work the text
Concern with the meaning and the substance of the content to extract actual information.

Hint: Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.

In [141]:
def convert_text(text):
   
    sentence = nlp(text)
    ents = {x.text: x for x in sentence.ents}
    tokens = []
    for w in sentence:
        if w.is_punct or w.is_stop:
            continue
        if w.text in ents:
            tokens.append(w.text)
        else:
            tokens.append(w.lemma_.lower())
    text = ' '.join(tokens)

    return text



In [142]:
%%time
df['short'] = df['Message'].apply(convert_text)

CPU times: user 9.52 s, sys: 13.3 ms, total: 9.53 s
Wall time: 9.54 s


In [94]:
df.sample(10)

Message  Target  \
Amazon 974  this phone tries very hard to do everything bu...       0   
IMDB   676  paolo sorrentino has written a wonderful story...       1   
       459  macbeth jason connery moved me to tears with h...       1   
Amazon 700  also the phone doesnt seem to accept anything ...       0   
IMDB   149   i love it  \t\nthe ending is so so perfect yo...       0   
       346  its just adorable seeing mickey playing turkey...       1   
       102  i dont think ive ever gone to a movie and disl...       0   
Yelp   876       ive had better bagels from the grocery store       0   
Amazon 116  i got this phone around the end of may and im ...       0   
Yelp   543      i wont try going back there even if its empty       0   

                                                        short  
Amazon 974                  phone try hard fail ability phone  
IMDB   676  paolo sorrentino write wonderful story lonelin...  
       459  macbeth jason connery move tear final monolog ...  
Amazon 700  phone nt accept cbr mp preferably rip window m...  
IMDB   149    love  \t\n ending perfect ll shed tear  \t\n...  
       346  adorable see mickey playing turkey straw highl...  
       102                     nt think ve go movie dislike    
Yelp   876                        ve well bagel grocery store  
Amazon 116                 get phone end m completely unhappy  
Yelp   543                                       wo nt try go

## Modelling

In [98]:
# helper function to show results and charts
def show_summary_report(actual, prediction):

    if isinstance(actual, pd.Series):
        actual = actual.values
    if actual.dtype.name == 'object':
        actual = actual.astype(int)
    if prediction.dtype.name == 'object':
        prediction = prediction.astype(int)

    accuracy_ = accuracy_score(actual, prediction)
    precision_ = precision_score(actual, prediction)
    recall_ = recall_score(actual, prediction)
    roc_auc_ = roc_auc_score(actual, prediction)

    print('Accuracy : %.4f [TP / N] Proportion of predicted labels that match the true labels. Best: 1, Worst: 0' % accuracy_)
    print('Precision: %.4f [TP / (TP + FP)] Not to label a negative sample as positive.        Best: 1, Worst: 0' % precision_)
    print('Recall   : %.4f [TP / (TP + FN)] Find all the positive samples.                     Best: 1, Worst: 0' % recall_)
    print('ROC AUC  : %.4f                                                                     Best: 1, Worst: < 0.5' % roc_auc_)
    print('-' * 107)
    print('TP: True Positives, FP: False Positives, TN: True Negatives, FN: False Negatives, N: Number of samples')

    # Confusion Matrix
    mat = confusion_matrix(actual, prediction)

    # Precision/Recall
    precision, recall, _ = precision_recall_curve(actual, prediction)
    average_precision = average_precision_score(actual, prediction)
    
    # Compute ROC curve and ROC area
    fpr, tpr, _ = roc_curve(actual, prediction)
    roc_auc = auc(fpr, tpr)

    # plot
    fig, ax = plt.subplots(1, 3, figsize = (18, 6))
    fig.subplots_adjust(left = 0.02, right = 0.98, wspace = 0.2)

    # Confusion Matrix
    sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False, cmap = 'Blues', ax = ax[0])

    ax[0].set_title('Confusion Matrix')
    ax[0].set_xlabel('True label')
    ax[0].set_ylabel('Predicted label')
    
    # Precision/Recall
    step_kwargs = {'step': 'post'}
    ax[1].step(recall, precision, color = 'b', alpha = 0.2, where = 'post')
    ax[1].fill_between(recall, precision, alpha = 0.2, color = 'b', **step_kwargs)
    ax[1].set_ylim([0.0, 1.0])
    ax[1].set_xlim([0.0, 1.0])
    ax[1].set_xlabel('Recall')
    ax[1].set_ylabel('Precision')
    ax[1].set_title('2-class Precision-Recall curve')

    # ROC
    ax[2].plot(fpr, tpr, color = 'darkorange', lw = 2, label = 'ROC curve (AUC = %0.2f)' % roc_auc)
    ax[2].plot([0, 1], [0, 1], color = 'navy', lw = 2, linestyle = '--')
    ax[2].set_xlim([0.0, 1.0])
    ax[2].set_ylim([0.0, 1.0])
    ax[2].set_xlabel('False Positive Rate')
    ax[2].set_ylabel('True Positive Rate')
    ax[2].set_title('Receiver Operating Characteristic')
    ax[2].legend(loc = 'lower right')

    plt.show()
    
    return (accuracy_, precision_, recall_, roc_auc_)

In [100]:
# Features and Labels
X = df['short']
y = df['Target']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Use Bag-of-Words

In [101]:
# create a matrix of word counts from the text
counts = CountVectorizer()

In [102]:
# do the actual counting
A = counts.fit_transform(X_train, y_train)

In [114]:
# create a classifier using SVC
classifier = SVC(kernel='linear', probability=True)

In [115]:
# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

SVC(kernel='linear', probability=True)

In [116]:
# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
B = counts.transform(X_test)

In [117]:
# make predictions based on the test data
predictions = classifier.predict(B)
predictions

ValueError: cannot use sparse input in 'SVC' trained on dense data

In [118]:
# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, predictions))

NameError: name 'predictions' is not defined

## Repeat using TF-IDF
TF-IDF = Term Frequency - Inverse Document Frequency

In [120]:
# create a matrix of word counts from the text
# use TF-IDF
tfidf = TfidfVectorizer()
# do the actual counting
A = tfidf.fit_transform(X_train, y_train)

# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
B = tfidf.transform(X_test)

# make predictions based on the test data
predictions = classifier.predict(B)

# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, predictions))

ValueError: cannot use sparse input in 'SVC' trained on dense data

## Repeating it all for comparision
Repeating the whole lot in one big block

Find 'Accuracy', 'Precision', 'Recall', 'ROC_AUC' using CountVectorizer and TfidfVectorizer and keep the result in a dataframe.

In [ ]:
# ANSWER



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



